# Week 4 lecture notes - Basic SQL DML

This week and the next few weeks we will be using our notebooks to connect to relational database systems.  To do this, we need the [ipython-sql](https://github.com/catherinedevlin/ipython-sql) library installed. It's already installed on your DMFA EC2 instance. If you are running the notebook on your DMFA EC2 instance, you are skip the rest of this section. 

To check, run this:

In [1]:
!pip freeze | grep ipython-sql

ipython-sql==0.3.9


## Part 1 - Data manipulation (DML)

Get the ```survey.db``` SQLite3 database file from the [Software Carpentry lesson](http://swcarpentry.github.io/sql-novice-survey/discussion.html) and connect to it.

In [2]:
!wget -O survey.db http://files.software-carpentry.org/survey.db

--2018-09-16 21:43:05--  http://files.software-carpentry.org/survey.db
Resolving files.software-carpentry.org (files.software-carpentry.org)... 52.84.122.73, 52.84.122.59, 52.84.122.204, ...
Connecting to files.software-carpentry.org (files.software-carpentry.org)|52.84.122.73|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.software-carpentry.org/survey.db [following]
--2018-09-16 21:43:06--  https://files.software-carpentry.org/survey.db
Connecting to files.software-carpentry.org (files.software-carpentry.org)|52.84.122.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20480 (20K) [application/octet-stream]
Saving to: ‘survey.db’

survey.db           100%[===================>]  20.00K  --.-KB/s    in 0s      

2018-09-16 21:43:06 (114 MB/s) - ‘survey.db’ saved [20480/20480]



In [3]:
%load_ext sql

In [4]:
%sql sqlite:///survey.db

'Connected: @survey.db'

## Basic queries: SELECT, DISTINCT, FROM, ORDER BY

Let's look at the data first:

In [7]:
%%sql
SELECT tbl_name FROM sqlite_master WHERE type='table';

 * sqlite:///survey.db
Done.


tbl_name
Person
Site
Visited
Survey


In [8]:
%%sql
SELECT * FROM person;

 * sqlite:///survey.db
Done.


ident,personal,family
dyer,William,Dyer
pb,Frank,Pabodie
lake,Anderson,Lake
roe,Valentina,Roerich
danforth,Frank,Danforth


In [9]:
%%sql
SELECT * FROM Site;

 * sqlite:///survey.db
Done.


name,lat,long
DR-1,-49.85,-128.57
DR-3,-47.15,-126.72
MSK-4,-48.87,-123.4


In [10]:
%%sql
SELECT * FROM Visited;

 * sqlite:///survey.db
Done.


ident,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
734,DR-3,1939-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
752,DR-3,None
837,MSK-4,1932-01-14
844,DR-1,1932-03-22


In [11]:
%%sql
SELECT * FROM Survey;

 * sqlite:///survey.db
Done.


taken,person,quant,reading
619,dyer,rad,9.82
619,dyer,sal,0.13
622,dyer,rad,7.8
622,dyer,sal,0.09
734,pb,rad,8.41
734,lake,sal,0.05
734,pb,temp,-21.5
735,pb,rad,7.22
735,None,sal,0.06
735,None,temp,-26.0


Simple queries involve a projection of attributes and a product of relations (here just one relation):

In [17]:
%%sql
SELECT personal, family
FROM Person

 * sqlite:///survey.db
Done.


personal,family
William,Dyer
Frank,Pabodie
Anderson,Lake
Valentina,Roerich
Frank,Danforth


Note that we can change the order of projected attributes:

In [13]:
%%sql
SELECT family, personal
FROM Person

 * sqlite:///survey.db
Done.


family,personal
Dyer,William
Pabodie,Frank
Lake,Anderson
Roerich,Valentina
Danforth,Frank


### Handling duplicates

Note that the `survey` table has a number of repeating values for the `quant` attribute.

In [14]:
%%sql
SELECT quant
FROM survey

 * sqlite:///survey.db
Done.


quant
rad
sal
rad
sal
rad
sal
temp
rad
sal
temp


By modifying our `SELECT` clause to use `DISTINCT quant` instead of just `quant`, we are assured of seeing each value only once.

In [15]:
%%sql
SELECT DISTINCT quant
FROM Survey

 * sqlite:///survey.db
Done.


quant
rad
sal
temp


They are radiation, salinity and temperature respectively. 

We can use `DISTINCT` on multiple attributes too.

In [20]:
%%sql
SELECT DISTINCT quant, person 
FROM Survey;

 * sqlite:///survey.db
Done.


quant,person
rad,dyer
sal,dyer
rad,pb
sal,lake
temp,pb
sal,None
temp,None
rad,lake
temp,lake
sal,roe


We can change the ordering of the resulting relation. Ordering is `ASC`ending by default; `DESC`ending is also easily specified, and each attribute can use either.

In [24]:
%%sql
SELECT person Person, reading as old_reading, reading+1  as "new reading"
FROM Survey;

 * sqlite:///survey.db
Done.


Person,old_reading,new reading
dyer,9.82,10.82
dyer,0.13,1.13
dyer,7.8,8.8
dyer,0.09,1.09
pb,8.41,9.41
lake,0.05,1.05
pb,-21.5,-20.5
pb,7.22,8.219999999999999
None,0.06,1.06
None,-26.0,-25.0


In [19]:
%%sql
SELECT taken, person, quant
FROM Survey 
ORDER BY taken ASC, person DESC;

 * sqlite:///survey.db
Done.


taken,person,quant
619,dyer,rad
619,dyer,sal
622,dyer,rad
622,dyer,sal
734,pb,rad
734,pb,temp
734,lake,sal
735,pb,rad
735,None,sal
735,None,temp


## Filtering: WHERE

Though it might sound confusing, the `WHERE` clause in SQL queries implements the selection operation.

In [ ]:
%%sql
SELECT * 
FROM Visited
WHERE site='DR-1'

Above the `site` attribute is included implicitly with `*`; here we project it explicitly.

In [ ]:
%%sql
SELECT ident 
FROM Visited 
WHERE site='DR-1';

Keep in mind the broader picture of what's happening here.  The following image, from the SW Carpentry lessons, illustrates the above query in terms of its fundamental select and project operations.

In [25]:
from IPython.display import Image
Image(url="http://swcarpentry.github.io/sql-novice-survey/fig/sql-filter.svg")

In selections, we can use boolean operators like `OR` and `AND` to combine conditions logically.

In [26]:
%%sql
SELECT * 
FROM Survey 
WHERE person='lake' 
  OR person='roe'

 * sqlite:///survey.db
Done.


taken,person,quant,reading
734,lake,sal,0.05
751,lake,sal,0.1
752,lake,rad,2.19
752,lake,sal,0.09
752,lake,temp,-16.0
752,roe,sal,41.6
837,lake,rad,1.46
837,lake,sal,0.21
837,roe,sal,22.5
844,roe,rad,11.25


We can test set membership in selections as well.

These boolean operations can nest into complex expressions using parentheses.

In [27]:
%%sql
SELECT * 
FROM Survey 
WHERE quant='sal' 
  AND (person='lake' OR person='roe')

 * sqlite:///survey.db
Done.


taken,person,quant,reading
734,lake,sal,0.05
751,lake,sal,0.1
752,lake,sal,0.09
752,roe,sal,41.6
837,lake,sal,0.21
837,roe,sal,22.5


Wildcards like `%` and `_` allow matching of substrings or single characters.

In [28]:
%%sql
SELECT * 
FROM Visited
WHERE site LIKE 'DR%';

 * sqlite:///survey.db
Done.


ident,site,dated
619,DR-1,1927-02-08
622,DR-1,1927-02-10
734,DR-3,1939-01-07
735,DR-3,1930-01-12
751,DR-3,1930-02-26
752,DR-3,None
844,DR-1,1932-03-22


We can start to put all of these together.

In [29]:
%%sql
SELECT DISTINCT person, quant 
FROM Survey 
WHERE person='lake' 
   OR person='roe';

 * sqlite:///survey.db
Done.


person,quant
lake,sal
lake,rad
lake,temp
roe,sal
roe,rad


Mathematical transformations are easy to compute on projected attributes.  Note also that we are here renaming an attribute with `AS`, creating a new alias.

In [30]:
%%sql
SELECT reading, 1.05 * reading AS reading_multiplied
FROM Survey 
WHERE quant='rad'

 * sqlite:///survey.db
Done.


reading,reading_multiplied
9.82,10.311
7.8,8.19
8.41,8.8305
7.22,7.581
4.35,4.5675
2.19,2.2995
1.46,1.533
11.25,11.8125


Sometimes we do this for convenience or clarity.

In [ ]:
%%sql
SELECT taken, reading AS reading_fahrenheit, round(5*(reading-32)/9, 2) AS reading_celsius
FROM Survey 
WHERE quant='temp'

Sometimes we even do this to make data redundant for clarity.  Here `||` is a concatenation operator.

In [ ]:
%%sql
SELECT personal || ' ' || family AS full_name, personal, family
FROM Person
ORDER BY family, personal

We can combine two relations with `UNION`.

In [ ]:
%%sql
SELECT * 
FROM Person 
WHERE ident='dyer' 

UNION 

SELECT * 
FROM Person 
WHERE ident='roe'

## NULL values

NULLs can be confusing but they have important qualities to remember.

In [ ]:
%%sql
SELECT * 
FROM Visited

Above we see `NULL` represented as `None` in the sqlite3 relation output.

Comparisons with NULL are NULL; thus boolean operations like the following will not match any NULL values.

In [ ]:
%%sql
SELECT * 
FROM Visited
WHERE dated < '1930-01-01'
  OR dated >= '1930-01-01'

In [31]:
%%sql
SELECT * 
FROM Visited 
WHERE dated = NULL;

 * sqlite:///survey.db
Done.


ident,site,dated


We have to use the special `IS NULL` and `IS NOT NULL` to deal explicitly with the concept of NULL.

In [32]:
%%sql
SELECT * FROM Visited WHERE dated IS NULL;

 * sqlite:///survey.db
Done.


ident,site,dated
752,DR-3,None


In [ ]:
%%sql
SELECT * FROM Visited WHERE dated IS NOT NULL;

Note the difference in the two following output results.

In [ ]:
%%sql
SELECT * 
FROM Survey 
WHERE quant = 'sal' 
  AND person != 'lake';

In [ ]:
%%sql
SELECT * 
FROM Survey
WHERE quant = 'sal'
  AND (person != 'lake' OR person IS NULL);

## Next Lecture
## Join multiple tables

Let's join `Site` and `Visited` tables.

In [ ]:
%%sql
SELECT *
FROM Site, Visited

How to fix it?

In [ ]:
%%sql
SELECT *
FROM Site, Visited
WHERE Site.name = Visited.site

Since the values of `name` and `site` columns are identical, we can remove one of them.

In [ ]:
%%sql
SELECT Site.name, Site.lat, Site.long,
       Visited.ident, Visited.dated
FROM Site, Visited
WHERE Site.name = Visited.site

## Aggregation: min(), max(), count(), avg()

Aggregation takes multiple input attribute values and returns a single output value based on the operation.

In [ ]:
%%sql
SELECT MIN(dated)
FROM Visited

This diagram, also from the SW Carpentry lessons, illustrates the process clearly.

In [ ]:
Image(url="http://swcarpentry.github.io/sql-novice-survey/fig/sql-aggregation.svg")

Each of these aggregators does what you think!

In [ ]:
%%sql
SELECT AVG(reading)
FROM Survey 
WHERE quant='sal'

In [ ]:
%%sql
SELECT COUNT(reading) 
FROM Survey 
WHERE quant='sal'

In [ ]:
%%sql
SELECT SUM(reading)
FROM Survey
WHERE quant='sal'

We can use more than one aggregation with attributes.

In [ ]:
%%sql
SELECT MIN(reading), MAX(reading)
FROM Survey 
WHERE quant = 'sal' 
  AND reading <= 1.0

## Aggregation: GROUP BY, HAVING

Aggregation allows the `GROUP BY` function that separates input attribute values into categories.

In [ ]:
%%sql
SELECT   person, COUNT(reading), ROUND(AVG(reading), 2)
FROM     Survey
WHERE    quant = 'rad'
GROUP BY person

We can add a special form of selection to grouped category attributes by using `HAVING` with `GROUP BY`.

In [ ]:
%%sql
SELECT   person, COUNT(reading), ROUND(AVG(reading), 2) AS avg_reading
FROM     Survey
WHERE    quant = 'rad'
GROUP BY person
HAVING   avg_reading < 10.0
ORDER BY person